<img align="right" width="120" height="120" style='padding: 0px 30px;' src="images/RR-logo.png"/>

# Introduction to Machine Learning with scikit-learn
This notebook will introduce the popular (and free!) machine learning toolkit [``scikit-learn``](https://scikit-learn.org/) written in Python. ``scikit-learn`` offers algorithms for supervised and unsupervised learning, dimensionality reduction, model selection and evaluation, and even some techniques for the visualization of results. It is designed to integrate nicely with other popular toolkits like [matplotlib](https://matplotlib.org/) and [plotly](https://plotly.com/) (for visualizations), [pandas](https://pandas.pydata.org/) (for tabular data analysis and manipulation), and [NumPy](https://numpy.org/) and [SciPy](https://scipy.org/) (for scientific computing). We will see some of this integration later!


In this exercise, we will walk through a typical processing pipeline to train and evaluate a simple classifier. 




*Click into a cell and hit "Shift+Enter" to execute it!*

In [ ]:
""" Let's apply some Dartmouth-style colors to our plots """

import matplotlib as mpl

dartmouth_colors = ["#00693E", "#12312B", "#C3DD88", "#6EAA8D", "#797979", "#EBF3EF"]

mpl.rcParams.update({
                        'figure.facecolor': "#EBF3EF",
                        'figure.figsize': [7.50, 3.50],
                        'axes.prop_cycle': mpl.cycler(color=dartmouth_colors),
                        'axes.facecolor': "#FFFFFF",
                        'axes.labelcolor': '#12312B',
                        'text.color': '#12312B'
                    })



## Loading a dataset

In this notebook, we will work with the famous *Iris* flower dataset. It is a multivariate dataset consisting of 50 samples from each of three species of *Iris*. Each sample is described in terms of four features: the length and width of its sepals and its petals.

<style type="text/css" >
table {
    border-collapse: collapse;
    text-align: center;
    border-top: 3px solid;
    border-bottom: 3px solid;
}

tr, td, th {
    border-bottom: none !important;
    border-left: none !important;
    border-right: none !important;
}

</style>

<table>
  <tr>
    <th>Iris setosa</th>
    <th>Iris versicolor</th>
    <th>Iris virginica</th>    
  </tr>
  <tr>
    <td><img align="center" width="200" height="200" src="images/iris_setosa.jpg"/></td>
    <td><img align="center" width="200" height="200" src="images/iris_versicolor.jpg"/></td>
    <td><img align="center" width="200" height="200" src="images/iris_virginica.jpg"/></td>
  </tr>
  <tr>
    <td><a href="https://commons.wikimedia.org/wiki/File:Iris_setosa_2.jpg" target="_blank">"Iris setosa"</a> by <a href="https://commons.wikimedia.org/wiki/User:Kulmalukko" target="_blank">Tiia Monto</a><br> is licensed under <a href="http://creativecommons.org/licenses/by-sa/4.0" target="_blank">CC BY-SA 4.0</a></td>
    <td><a href="https://commons.wikimedia.org/wiki/File:Blue_Flag_Iris_(15246206044).jpg" target="_blank">"Blue Flag Iris"</a> by <a href="https://www.flickr.com/people/49208525@N08" target="_blank">USFWSmidwest</a><br> is licensed under <a href="http://creativecommons.org/licenses/by/2.0" target="_blank">CC BY 2.0</a></td>
    <td><a href="https://commons.wikimedia.org/wiki/File:Iris_virginica_L_JdP_2013-05-28_n01.jpg" target="_blank">"Virginia Iris"</a> by <a>Marie-Lan Nguyen</a><br> is licensed under <a href="http://creativecommons.org/licenses/by/2.5" target="_blank">CC BY 2.5</a></td>    
  </tr>
  <tr>
    <td>Class 0</td>
    <td>Class 1</td>
    <td>Class 2</td>    
  </tr>
</table>

The data was originally collected by the American botanist Edgar Anderson [[1]](#anderson_1936). It is so well known today because of its use by the British statistician and biologist Ronald Fisher, who used it to showcase his method of Linear Discriminant Analysis [[2]](#fisher_1936). Because of the dataset's simplicity and interesting structure, it lends itself well to demonstrations of all sorts of algorithms and is therefore still popular today among machine learning educators.

It is in fact so popular that it is shipped with ``scikit-learn`` and can be easily loaded into our workspace using the function [``load_iris()``](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) from the submodule ``sklearn.datasets``. 

Because it offers loads of convenient methods to manipulate and visualize our tabular data, we want to load the dataset as a [``pandas.DataFrame``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) object. We can do this by setting the parameter ``as_frame`` to ``True``.

In [ ]:
from sklearn.datasets import load_iris

dataset = load_iris(as_frame=True)

When we load the dataset in this way, we get a special, ``scikit-learn``-specific data structure called a [``Bunch``](https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html#sklearn.utils.Bunch). A ``Bunch`` is very much like a standard Python [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries): It stores pairs of *keys* and *values*. We can obtain the *values* by passing the *key*. We can get a list of all the keys in the ``Bunch`` by calling its ``keys()`` method:

In [ ]:
dataset.keys()

Feel free to explore the values corresponding to the other keys, but in this notebook we will simply grab the ``DataFrame`` that is stored under ``frame`` and the names of the features (``'feature_names'``) and species (``'target_names'``):

In [ ]:
iris = dataset['frame']
target_names = dataset['target_names']
feature_names = dataset['feature_names']

<div class="alert alert-block alert-info">

<b>Quick sidenote:</b> A `DataFrame` is an object defined in the package `pandas`! `scikit-learn` uses it, because it is a great way to represent the tabular data. This is an example of the integration with other packages mentioned above. Why re-invent the wheel if you can build upon the great work of others? After all, we are basically doing the same thing when we use `scikit-learn` in our own code!
</div>

Let's do some quick visualization to see how many samples of each species we have in our dataset (the *class distribution*):

In [ ]:
iris.target.value_counts().plot(kind='bar', rot=0)

Note that we did not call any `scikit-learn` code here, but `DataFrame` methods, which are implemented in `pandas`! But we don't really need to worry about that, because everything is so neatly integrated.

Great, so we have the same number of samples from each species (or class). That means our dataset is *balanced*. This is a good thing because in an *imbalanced* dataset, machine learning models tend to be better at identifying the majority class than at identifying the minority class (i.e., they exhibit high bias) and we need to apply special techniques to counter this behavior (e.g. over- or undersampling, or special scoring metrics). Our balanced Iris dataset does not need any special attention in that regard.

Next, let us take a look a the features and if we can spot any patterns in them!

One way to do this is to plot all the two-dimensional scatter plots:
- *sepal length* versus *sepal width*
- *petal length* versus *petal width*
- *sepal length* versus *petal length*
- ...

That is a lot of pairwise plots. Luckily, we can use a function from the package [``seaborn``](https://seaborn.pydata.org/index.html) called [``pairplot()``](https://seaborn.pydata.org/generated/seaborn.pairplot.html) that conveniently does all the work for us!

`seaborn` is a library for statistical data visualization that plays very nicely with `DataFrame` objects. So let's import and use it:

In [ ]:
import seaborn as sns
sns.pairplot(data=iris, hue='target', palette=sns.color_palette(dartmouth_colors)[:3])

We can see that in some plots, dots of the same color are neatly clustered and the clusters overlap very little. Others are a bit more messy. 

<div class="alert alert-block alert-info">

**What do you think:** Which species (or *class*) may be harder to distinguish from the other two?
</div>

We also suspect some strong correlations between features. Let's investigate by calculating the pair-wise correlation using the ``DataFrame`` method [``corr()``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)!

In [ ]:
iris[feature_names].corr()

Indeed, we have quite strongly correlated features in this dataset! 

In machine learning, we generally prefer uncorrelated features because correlated features add only very little information that can help to differentiate between the classes.

We should keep the strong correlations in mind for later. As a general rule, a simple model using few features is preferred over a complex model using many features. So maybe we don't even need to use all features in our model?



## Preprocessing


A common issue when dealing with a multivariate dataset is that the different features can be on very different scales. That means that when we compute distances in the feature space, features with generally high values will contribute disproportionately to the overall distance.

Before we process our data further, it is therefore usually a good idea to normalize or standardize our data. A common standardization method is to calculate the *$z$-score*. We subtract the mean $\mu$  from each feature value $x$ and divide it by the standard deviation $\sigma$: 

$$z = \frac{x - \mu}{\sigma}$$

This transformation does not change the relative distribution of the data, it only transforms the *scale*. Here is an example of two samples, both with the same relative distribution but on different scales (left column). The shape of the distribution does not change when we calculate their respective $z$-scores (right column). But now they are on the same scale!

<center><img src=images/zscore_histograms.svg></center>

Let's see what this means for three individual observations $A$, $B$, and $X$ in our sample:

<center><img src="images/zscore_distance_annotated.svg"></center>

In machine learning, we usually use distance metrics defined in the feature space to express how similar or dissimilar two observations are. If we use the original scale, observation $A$ would have a Euclidean distance of $225$ from observation $X$, and $B$ would have a distance of $2.25$. You would thus conclude that $A$ is much less similar to $X$ than $B$. But as we can see in the figure above, the different scales distort our metric! When using the $z$-scores, which standardize the features to their dispersion, the distances in this case indeed become equal.

A $z$-score of 1 means *1 standard deviation from the mean* on the original scale. As you can see, this makes distance calculations much less biased and therefore more meaningful!


Conveniently, `scikit-learn` offers a [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) class to make the scaling process very easy:

In [ ]:
from sklearn.preprocessing import StandardScaler

X = iris[feature_names]
y = iris['target']

scaler = StandardScaler()       # Instantiate the scaler
scaler.fit(X)                   # Fit the scaler on the data and calculate its parameters (mean and variance)
print(f'The feature means are {scaler.mean_} and their variances are {scaler.var_}')

X_scaled = scaler.transform(X)  # Transform the data (using the previously calculated mean and variance)
print(f'After the transformation, the feature means are {X_scaled.mean(axis=0)} and their variances {X_scaled.var(axis=0)}')

Note that using the `StandardScaler` is a three step sequence:

<div class="alert alert-block alert-info">

**Core workflow**
1. Instantiate the object
2. Fit the object's parameters on the data
3. Transform the data using the fitted object
</div>

This basic pattern is the core workflow in `scikit-learn` and we will see it over and over again in the rest of this notebook!

## Dimensionality reduction

As we have discovered above, some of the features are quite strongly correlated. It may therefore be helpful to decorrelate them using [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis). Let us investigate the effect of PCA on our data by starting with just two features to be able to visualize what is going on.

Let's take a look at petal width and petal length again to remind ourselves of the strong correlation. When we used the `transform()` method of the `StandardScaler`, we also changed the type of our variable from `pandas.DataFrame` to  a `numpy.array`. So yet another package being used! `numpy.array`s are very efficient regarding numerical computations, but they unfortunately do not offer all the nice visualization options that `pandas` and `seaborn` have. Instead, we need to bring in another package called `matplotlib` to create a scatter plot of our data:

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(X_scaled[:, 2], X_scaled[:, 3])
plt.xlabel('$z$-score of petal length')
plt.ylabel('$z$-score of petal width');

Note the upward trend indicating a strong correlation between the features! We will now try to get rid of this by using PCA:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()                             # Instantiate
pca.fit(X_scaled[:, 2:4])               # Fit

X_pc = pca.transform(X_scaled[:, 2:4])  # Transform (here, we only transform a subset of our features for illustration purposes)

plt.scatter(X_pc[:, 0], X_pc[:, 1])
plt.xlabel('PC 1 of petal length and width')
plt.ylabel('PC 2 of petal length and width')
plt.ylim(plt.xlim())
plt.tight_layout()

Notice how the upward trend is almost gone? Let's check the correlation before and after:

In [ ]:
import numpy as np
rho = np.corrcoef(X_scaled[:, 2], X_scaled[:, 3])[0, 1]
print(f"Before the PCA transform, the two features have a correlation coefficient of {rho = :.2f}.")

rho = np.corrcoef(X_pc[:, 0], X_pc[:, 1])[0, 1]
print(f"After the PCA transform, the two features have a correlation coefficient of {rho = :.2f}.")

Notice also how the variance of the second PC is a lot less than the one of the first PC:

In [ ]:
print(f"The first PC has a variance of {X_pc[:, 0].var():.2f}.")
print(f"The second PC has a variance of {X_pc[:, 1].var():.2f}.")

A feature with an almost constant value is not really useful to a machine learning algorithm, because it is the same for every class and does not add any information that helps to discriminate between the classes. We could therefore simply remove this dimension now and only continue with the first principal component!

But first, let's transform all features into the principal components and keep them all for now. We will come back to this later.

In [ ]:
pca = PCA()                     # Instantiate
pca.fit(X_scaled)               # Fit

X_pc = pca.transform(X_scaled)  # Transform

There are, of course, a lot more things we could do to preprocess our data. If you like, take a look at the [`sklearn.preprocessing` submodule](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing), pick one of the methods, and try to apply it to our data. For the purposes of this notebook, however, we will consider the preprocessing complete at this point.

## Training a classifier

Now that our data is properly preprocessed, it is time to train a classifier to identify our flowers' species based on their recorded features. Classifiers behave similar to the `StandardScaler` and the `PCA` objects we have already used. To understand their overlap and differences, it may help to briefly discuss `scikit-learns` [basic building blocks](https://scikit-learn.org/stable/developers/develop.html):

<figure style="float:right;width:40%">

<img src="https://scikit-learn.org/stable/_static/ml_map.png">

<figcaption>Source: <a href="https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html">Scikit-learn user guide: Choosing the right estimator</a></figcaption>
</figure>

### Estimators
Almost all objects in `scikit-learn` are some kind of `Estimator`. An `Estimator` implements a `fit` method to learn its parameters from data. `StandardScaler`, `PCA`, all classifiers and regression models, and many other objects (some of which we will see later) are (also) `Estimator`s.

### Transformer
A `Transformer` is an `Estimator` that implements a `transform` method to modify or filter data. Both `PCA` and `StandardScaler` are `Transformer`s. Most classifiers or regressors are not `Transformer`s, because they do not modify the data.

### Predictor
A `Predictor` is an `Estimator` that also implements a `predict` method to make a prediction based on observed data. This method can generally only be called after `fit` was called. All classifiers and regressors are `Predictor`s.

### Model
A model is a `Predictor` that also implements a `score` method to calculate some kind of [goodness-of-fit](https://en.wikipedia.org/wiki/Goodness_of_fit) or accuracy metric for the predictions.

So there you have it: Preprocessing steps usually involve objects that are some kind of `Transformer`, while the final step of classifying (or fitting a regression model to) the data is done by some kind of `Predictor` or, more commonly, `Model`.

There is a vast array of models available in `scikit-learn` and choosing the best one for your specific problem is a challenging task all by itself. Recommendations that often look like flow-charts (e.g. the one on the right) exist, but they can only give you a good starting point. Finding the truly optimal model requires implementing, training, tuning, and testing a large number of candidates. Luckily, `scikit-learn`'s neat interface lets us do this fairly quickly and efficiently by following the same workflow for all models.

One of the fundamental steps in this workflow is training a model. In keeping with the pattern we already applied in the preprocessing, we train a model by *instantiating* an object and then *fitting* its parameters on the data.

For this notebook, we will pick a [$k$-nearest neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) classifier because it is conceptually simple and will help us focus on the workflow instead of the internals of the model.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()    # Instantiate

knn.fit(X_pc, y)                # Fit

<div class="alert alert-block alert-info">

You might wonder where exactly the training happens. Note that *fitting* means *learning the models parameters from the data*. That is exactly what we mean by training. So don't expect models to define a `train` method: In `scikit-learn`, this is called `fit`!
</div>

## Testing an estimator
After training the classifier, you probably want to know how accurate its predictions are. Can it really tell the different species of iris apart now? Let's find out!

Your first intuition may be to use the classifier we have trained (or fitted) above and compare its predictions to the known, ground truth labels.

In [ ]:
y_pred = knn.predict(X_pc)  # Predict a label (species code) for each observation

(y_pred == y).value_counts()


To quantify this comparison, we want to use a so-called *scoring* algorithm of some kind. `scikit-learn` gives us a lot of [options to choose from](https://scikit-learn.org/stable/modules/model_evaluation.html) and we could also write our own. For this example we will simply use the *accuracy* score, which is defined as:

$$
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}}
$$

In [ ]:
from sklearn.metrics import accuracy_score

print(f"The classifier's accuracy is {accuracy_score(y_pred=y_pred, y_true=y)*100:.2f} %.")

Well, that certainly looks great. But we made a grave mistake: The model is scored on the same data we used during training! 

Since our example $k$-nearest neighbor classifier stores all training observations and their respective labels, it should be able to achieve a perfect score. 

<div class="alert alert-block alert-info">

Can you say why it is only achieving an *almost* perfect score in our example here, even though it has all the observations and their labels in memory? 

**Hint:** Check <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">the model's default options</a>.
</div>

A better way to test how our model would behave when it has to label new observations *unseen* during training, is to hold out some of our data from the training and then use it to test. 

To facilitate this so-called *hold-out validation*, `scikit-learn` provides a function called `train_test_split` to split a dataset randomly into two subsets.

We can specify the desired relative size of the training set or of the test set. Let's split our original dataset and mark 80 % for training and 20 % for testing:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=8)  # The random state is set here to ensure reproducible results

Let's train our model using *only* the data in training set. Don't forget the preprocessing:

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)  # You can conveniently fit and transform in a single step using this method
X_train_pc = pca.fit_transform(X_train_scaled)

knn.fit(X_train_pc, y_train)

<div class="alert alert-block alert-info">

Why do you think we split the original dataset instead of doing the preprocessing first and then doing the split?

**Hint:** It has something to do with the concept of [data leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning))!
</div>

Our model is now trained and we can get a good idea of its ability to generalize its knowledge gained from the training set to new observations by calculating the accuracy on the test set:

In [ ]:
# Preprocessing
X_test_scaled = scaler.transform(X_test)
X_test_pc = pca.transform(X_test_scaled)

# Predictions
y_pred = knn.predict(X_test_pc)

print(f"The classifier's accuracy on the training set is {accuracy_score(y_pred=y_pred, y_true=y_test)*100:.2f} %.")

As we would expect, the accuracy on unseen data is lower, but it is still overall fairly high. We can get a more detailed look at the strengths and weaknesses of our model by printing a [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html):

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=target_names))

The report nicely breaks down the overall accuracy by class and into [*precision*, *recall*,](https://en.wikipedia.org/wiki/Precision_and_recall) and [*F1 score*](https://en.wikipedia.org/wiki/F-score). It also lists the *support* for each class, which is the number of observations of this class included in the training set.

Another way to visualize the classification results is using a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix).

Dealing with a confusion matrix in `scikit-learn` can indeed get a bit confusing. If you need a confusion matrix as an actual matrix, you want to use the function [`confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). If you want to visualize the confusion matrix as a figure, you want [`ConfusionMatrixDisplay`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html).

Let's take a look at both:

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

""" All of the following produce the same visualization """

# Display the confusion matrix calculated above:
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names).plot()

# Calculate the confusion matrix from the true and predicted labels internally, then visualize
ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=y_pred, display_labels=target_names)

# Calculate the predicted labels and the resulting confusion matrix internally, then visualize
ConfusionMatrixDisplay.from_estimator(estimator=knn, X=X_test_pc, y=y_test, display_labels=target_names)

The confusion matrix tells us that two instances of irises were misclassified. Can we tweak our classifier to improve the results?

## Hyperparameter tuning

 We did not really spent any time discussing the various options you have in a $k$-nearest neighbors classifier, e.g.:

- Number of neighbors to consider (i.e., the value of $k$)
- How to calculate the distance/similarity
- How to choose the label based on the $k$ surrounding neighbors: Simple majority or weight each neighbor by its distance?
- ...

None of these options are learned directly from the data in the training step, that's why we have to specify them during the instantiation. These "un-learnable" options of a model are called *hyperparameters*.

Every hyperparameter we do not explicitly specify will use a [default value](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). These defaults are usually sensible starting points or based on common practice, but rarely optimal for your particular problem. We therefore have to *tune* them to find their most suitable values!

If you had to do it manually, the workflow to find the optimal hyperparameters would probably look something like this:

1. Pick a value for each hyperparameter.
2. Train and score the model.
3. Compare the score to the score achieved with the previous choice of hyperparameter values. If better, keep hyperparameter values.

With the code we have discussed in this notebook so far, you could already implement this strategy. But there is a more convenient and feature-rich way to do it using hyperparameter optimizers from the [`sklearn.model_selection`](https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers) module. 

All of these optimizers are based on a technique called $k$-fold cross-validation:

<figure style="width:60%;">
<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" style="padding:10px;background-color:white">
<figcaption>

$k$-fold cross-validation: 
1. Split the training data into $k$ folds. 
2. Use $k-1$ folds for training and the remaining fold for testing.
3. Repeat step 2, using a different fold for testing, until each fold was the test set once.
4. The final score is the average score across all folds.

Image Source: [`scikit-learn` user guide](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance)
</figcaption>
</figure>

Using the hyperparameter optimizers is based on the following pattern:

1. Define the candidate values for the hyperparameters (i.e. the *search space*)
2. Pick a set of hyperparameter values from the search space
3. Train and score a model using these hyperparameters
4. Repeat from step 2 until some stopping criterion is met
5. Return the set of hyperparameters that achieved the highest score

The differences between the optimizers lie in the strategy for picking a set of hyperparameters to evaluate:

To use `GridSearchCV`, we define a *parameter grid*, where every hyperparameter value is explicitly defined. This grid is then searched exhaustively (every possible combination will be tried).

To use `RandomizedSearchCV`, we define *distributions* for each hyperparameter. The search then samples from these distributions a fixed, user-specified number of times.

There are also `Halving` variants of these two basic strategies, which are beyond the scope of this introduction.

Which optimizer you should choose depends on the complexity of the search space. An exhaustive grid search will always identify the best combination of hyperparameter values. But if there are a lot of candidates to evaluate, the computation time may become unreasonably long. In that case it may be better to go with a randomized search, which will approximate the global optimum after a sufficient number of iterations.

Since our problem here is relatively simple, we will use the grid search:

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the search space as a parameter grid
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'weights': ['uniform', 'distance'],  
}

search = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=3, n_jobs=-1)     # Instantiate
search.fit(X_train_pc, y_train)                                                     # Fit

print(f"The best hyperparameter values were: {search.best_params_}")

In [ ]:
print(classification_report(y_test, search.predict(X_test_pc)))
ConfusionMatrixDisplay.from_estimator(estimator=search, X=X_test_pc, y=y_test, display_labels=target_names)

Even with the tuned hyperparameters, our classifier fails to classify two irises correctly. However, there are more options to our pipeline so far than the hyperparameters of the classifier itself. Maybe the scaling is not such a good idea after all? Maybe using fewer principal components would help?

These options could also be considered hyperparameters of our overall pipeline and therefore should be tuned together with the classifier's.

## Putting it all together

So we want to be able to find the optimal set of options not just for our classifier, but for our entire processing pipeline. For that exact purpose, `scikit-learn` provides the [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class!

A `Pipeline` is essentially a sequence of processing steps. Each step in the sequence is executed by a `Transformer` object, except for the final step, which is done by a `Model`.

So in terms of how the code flows, you could think of a `Pipeline`'s execution like the following pseudo-code example:

```python
pipeline = [transformer_1, transformer_2, transformer_3, model]
pipeline.fit(X, y)
```
This is equivalent to (read from the inside out):
```python
model.fit(
    transformer_3.fit_transform(
        transformer_2.fit_transform(
            transformer_1.fit_transform(X, y)
            )
        )
    )
```

The syntax to set up the pipeline is actually not that much different from the above:

In [ ]:
from sklearn.pipeline import Pipeline

# Define a list of N steps, where the first N-1 steps are Transformer objects and the final Nth step is a Model
steps = [
    ('scaling', StandardScaler()),      # Transformer
    ('pca', PCA()),                     # Transformer
    ('knn', KNeighborsClassifier())     # Model
]

pipeline = Pipeline(steps)              # Instantiate the Pipeline object

pipeline.fit(X_train, y_train)          # Fit 

y_pred = pipeline.predict(X_test)       # Predict (no need to preprocess X_test, it's all handled by the pipeline!)

print(f"The pipeline's accuracy on the training set is {accuracy_score(y_pred=y_pred, y_true=y_test)*100:.2f} %.")

We can now see the entire processing pipeline in just a view lines of code. Isn't that neat?

But wait, there's more: 

Let's take a step back and think about what the pipeline is in `scikit-learn` terms. It is an object that defines a `fit` method and a `predict` method. So it is a `Model`! And if it is a `Model`, we can use one of the hyperparameter optimizers to tune its hyperparameters!

We just need to figure out, how to define the parameter search space. This is a little more complicated than before because how should `scikit-learn` know, which step's hyperparameters we mean?

So we have to not just specify the names of the hyperparameters but also in which step they are relevant. When we set up the steps, we gave each step a name. This comes in handy now, because we simply prefix the name of the hyperparameter with the name of the corresponding step and a double underscore:

In [ ]:
param_grid = {
    'scaling__with_mean': [True, False],                            # Parameter named 'with_mean' of the transformer used in the step named 'scaling'
    'scaling__with_std': [True, False],                             # Parameter named 'with_std' of the transformer used in the step named 'scaling'
    'pca__n_components': [1, 2, 3, 4],                              # Parameter named 'n_components' of the transformer used in the step named 'pca'
    'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],            # ...
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knn__leaf_size': [10, 20, 30, 40, 50, 60, 70, 80, 90],
    'knn__p': [1, 2, 3]    
}

Now that this is taken care of, the rest works just as before:

In [ ]:
find_best = GridSearchCV(pipeline, param_grid, verbose=1, n_jobs=-1)

find_best.fit(X_train, y_train)

Now that our pipeline is fitted using the optimal hyperparameters, let's test it using the held-out test set:

In [ ]:
y_pred = find_best.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=y_pred, display_labels=target_names)

Wow, a perfect score!

## What have we learned
<figure style='float:right;width:25%'>
<img src="https://scikit-learn.org/stable/_images/grid_search_workflow.png" style='background-color:#EBF3EF;padding:10px'>
<figcaption>

A typical machine learning workflow (Source: [scikit-learn user guide](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance))
</figcaption>
</figure>

1. Machine learning is fun, but can involve lots of trial and error.
2. `scikit-learn` offers implementations of all sorts of algorithms and models using a consistent interface
3. Following `scikit-learn`'s interface conventions and making good use of the provided objects leads to very readable, concise, and efficient code

## Where to go from here

If this got you interested in `scikit-learn` or machine learning in general, you should check out [`scikit-learn`'s user guide](https://scikit-learn.org/stable/user_guide.html). It is an excellent resource not just regarding the library's use, but also with regards to algorithms, models, and workflows in machine learning.

You could also check out some libraries related to `scikit-learn`: 
* [`PyCaret`](https://pycaret.org/) is a higher-level, low-code library with a focus on automating the machine learning workflow even further!
* [`skorch`](https://skorch.readthedocs.io/en/latest/?badge=latest) is a `scikit-learn`-style wrapper around the neural network library [PyTorch](https://pytorch.org/).
* [`sktime`](https://www.sktime.org/en/stable/) is a framework for time series machine learning tasks.
* [`imbalanced-learn`](https://github.com/scikit-learn-contrib/imbalanced-learn) is a library focused on machine learning with imbalanced datasets.

Finally, the best way to learn is to solve some problems. You can find many datasets of varying complexity to work with on the data science platform [kaggle.com](https://www.kaggle.com/). Why not try to classify different [coffee types](https://www.kaggle.com/datasets/michals22/coffee-dataset)? Or play around with a [Hunger Games dataset](https://www.kaggle.com/datasets/thedevastator/the-hunger-games-dataset-a-survival-analysis)?

[1] <a id=anderson_1936></a>Edgar Anderson (1936). "The species problem in Iris". *Annals of the Missouri Botanical Garden*. 23 (3): 457–509. [doi:10.2307/2394164](https://doi.org/10.2307%2F2394164). JSTOR [2394164](https://www.jstor.org/stable/2394164).

[2] <a id=fisher_1936></a>R. A. Fisher (1936). "The use of multiple measurements in taxonomic problems". *Annals of Eugenics*. 7 (2): 179–188. [doi:10.1111/j.1469-1809.1936.tb02137.x](https://doi.org/10.1111%2Fj.1469-1809.1936.tb02137.x). hdl:[2440/15227](https://hdl.handle.net/2440%2F15227). 

<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>